Create a bootstrap confidence interval and a hypothesis test comparing the performance of two the models across all the data 

In [4]:
from scipy.stats import ttest_rel, wilcoxon
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel


In [5]:
test_predictions=pd.read_csv("test_predictions.csv")
test_predictions


,yearmonth,fips,y_pred_transformer,y_pred_proba_transformer,y_true_transformer,y_pred_xgboost,y_pred_proba_xgboost,y_true_xgboost,y_pred_ffnn,y_pred_proba_ffnn,y_true_ffnn,iso3
0,202211,FJ,False,0.183897,False,False,0.066500,False,False,0.409958,False,FJI
1,202212,FJ,False,0.267831,False,False,0.099643,False,False,0.406696,False,FJI
2,202211,TZ,False,0.482585,False,True,0.704086,True,True,0.545236,False,TZA
3,202212,TZ,False,0.187792,False,True,0.638444,True,True,0.534560,False,TZA
4,202301,TZ,True,0.539319,True,True,0.608380,False,True,0.538583,True,TZA
...,...,...,...,...,...,...,...,...,...,...,...,...
359,202211,MJ,False,0.182196,False,False,0.079453,False,False,0.291874,False,MNE
360,202212,MJ,False,0.203236,False,False,0.060189,False,False,0.300321,False,MNE
361,202211,TD,True,0.527107,False,True,0.697625,True,False,0.335496,False,TTO
362,202212,TD,True,0.555677,False,True,0.729246,False,False,0.324000,False,TTO


# H_0: two samples have the same mean
# H_1: two samples have the different mean

## transformer and xgboost p value and confidence interval ## 

In [ ]:
### permutation shuffling

In [6]:
df_t=test_predictions["y_pred_proba_transformer"]
df_x=test_predictions["y_pred_proba_xgboost"]
df_f=test_predictions["y_pred_proba_ffnn"]
df_tx=pd.concat([df_t,df_x],axis=0, ignore_index=True)
df_tf=pd.concat([df_t,df_f],axis=0, ignore_index=True)
df_fx=pd.concat([df_f,df_x],axis=0, ignore_index=True)

In [7]:
# Set the random seed
np.random.seed(109)

# Calculate the observed 
observed_test_statistics = df_t.mean() - df_x.mean()


simdata = df_tx.copy()


sim_test_statistics = []


for i in range(10000):
    # Shuffle the data randomly
    permu_df_tx = simdata.sample(frac=1).values

    # Split the shuffled data into two groups
    permu_df_t = permu_df_tx[0 : len(permu_df_tx) // 2]
    permu_df_x = permu_df_tx[len(permu_df_tx) // 2:]

    # Calculate the test statistic 
    sim_test_statistic = permu_df_t.mean() - permu_df_x.mean()

    # Store the test statistic in the list
    sim_test_statistics.append(sim_test_statistic)

# Count how many times the permuted test statistics are extreme or more extreme
num = np.sum(np.abs(sim_test_statistics) >= np.abs(observed_test_statistics))

# Calculate the p-value
p = np.round(num / 10000, 3)

confidence_interval = np.percentile(sim_test_statistics, [2.5, 97.5])
print(p,confidence_interval)

0.034 [-0.03089756  0.02969562]


In [ ]:
### 另外一个方法 

In [8]:

import pandas as pd
import numpy as np
from scipy.stats import ttest_rel


test_predictions = pd.read_csv("test_predictions.csv")

# Calculate paired differences
test_predictions['model_difference'] = test_predictions['y_pred_proba_transformer'] - test_predictions['y_pred_proba_xgboost']

# Set the number of bootstrap samples
n_bootstrap_samples = 1000

# Create an array to store bootstrap sample means
bootstrap_sample_means = np.zeros(n_bootstrap_samples)

for i in range(n_bootstrap_samples):
    # Generate a bootstrap sample of paired differences
    bootstrap_sample = np.random.choice(test_predictions['model_difference'], len(test_predictions), replace=True)
    
    # Calculate the sample statistic (mean) for the bootstrap sample
    bootstrap_sample_means[i] = np.mean(bootstrap_sample)

# Calculate the confidence interval
confidence_interval = np.percentile(bootstrap_sample_means, [2.5, 97.5])

# Perform a t-test on the paired differences
ttest_statistic, ttest_pvalue = ttest_rel(test_predictions['y_pred_proba_transformer'], test_predictions['y_pred_proba_xgboost'])
wilcoxon_pvalue = wilcoxon(test_predictions['y_pred_proba_transformer'], test_predictions['y_pred_proba_xgboost'])
confidence_interval, ttest_pvalue, wilcoxon_pvalue

(array([-0.05222857, -0.01358091]),
 0.001467421045216591,
 WilcoxonResult(statistic=26934.0, pvalue=0.0017683514481583043))

## transformer and ffnn ##

In [9]:
observed_test_statistics = df_t.mean() - df_f.mean()

# Create a copy of the original data for permutation
simdata = df_tf.copy()

# Initialize a list to store simulated test statistics
sim_test_statistics = []

# Perform 10,000 permutations
for i in range(10000):
    # Shuffle the data randomly
    permu_df_tf = simdata.sample(frac=1).values

    # Split the shuffled data into two groups
    permu_df_t = permu_df_tf[0 : len(permu_df_tf) // 2]
    permu_df_f = permu_df_tf[len(permu_df_tf) // 2:]

    # Calculate the test statistic for the permuted data
    sim_test_statistic = permu_df_t.mean() - permu_df_f.mean()

    # Store the test statistic in the list
    sim_test_statistics.append(sim_test_statistic)

# Count how many times the permuted test statistics are greater than or equal to the observed statistic
num = np.sum(np.abs(sim_test_statistics) >= np.abs(observed_test_statistics))

# Calculate the p-value
p = np.round(num / 10000, 3)

confidence_interval = np.percentile(sim_test_statistics, [2.5, 97.5])
print(p,confidence_interval)

0.0 [-0.02152488  0.02138699]


In [10]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel

# Load the data into a DataFrame
test_predictions = pd.read_csv("test_predictions.csv")

# Calculate paired differences
test_predictions['model_difference'] = test_predictions['y_pred_proba_transformer'] - test_predictions['y_pred_proba_ffnn']

# Set the number of bootstrap samples
n_bootstrap_samples = 1000

# Create an array to store bootstrap sample means
bootstrap_sample_means = np.zeros(n_bootstrap_samples)

for i in range(n_bootstrap_samples):
    # Generate a bootstrap sample of paired differences
    bootstrap_sample = np.random.choice(test_predictions['model_difference'], len(test_predictions), replace=True)
    
    # Calculate the sample statistic (mean) for the bootstrap sample
    bootstrap_sample_means[i] = np.mean(bootstrap_sample)

# Calculate the confidence interval
confidence_interval = np.percentile(bootstrap_sample_means, [2.5, 97.5])

# Perform a t-test on the paired differences
ttest_statistic, ttest_pvalue = ttest_rel(test_predictions['y_pred_proba_transformer'], test_predictions['y_pred_proba_ffnn'])
wilcoxon_pvalue = wilcoxon(test_predictions['y_pred_proba_transformer'], test_predictions['y_pred_proba_ffnn'])
confidence_interval, ttest_pvalue, wilcoxon_pvalue

(array([0.02485198, 0.05903583]),
 8.593371129451824e-06,
 WilcoxonResult(statistic=25852.0, pvalue=0.00024713024828037854))

## ffnn and xgboost ##

In [11]:
observed_test_statistics = df_f.mean() - df_x.mean()

# Create a copy of the original data for permutation
simdata = df_fx.copy()

# Initialize a list to store simulated test statistics
sim_test_statistics = []

# Perform 10,000 permutations
for i in range(10000):
    # Shuffle the data randomly
    permu_df_fx = simdata.sample(frac=1).values

    # Split the shuffled data into two groups
    permu_df_f = permu_df_fx[0 : len(permu_df_fx) // 2]
    permu_df_x = permu_df_fx[len(permu_df_fx) // 2:]

    # Calculate the test statistic for the permuted data
    sim_test_statistic = permu_df_f.mean() - permu_df_x.mean()

    # Store the test statistic in the list
    sim_test_statistics.append(sim_test_statistic)

# Count how many times the permuted test statistics are greater than or equal to the observed statistic
num = np.sum(np.abs(sim_test_statistics) >= np.abs(observed_test_statistics))

# Calculate the p-value
p = np.round(num / 10000, 3)

confidence_interval = np.percentile(sim_test_statistics, [2.5, 97.5])
print(p,confidence_interval)

0.0 [-0.02695939  0.02665751]


In [12]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel

# Load the data into a DataFrame
test_predictions = pd.read_csv("test_predictions.csv")

# Calculate paired differences
test_predictions['model_difference'] = test_predictions['y_pred_proba_xgboost'] - test_predictions['y_pred_proba_ffnn']

# Set the number of bootstrap samples
n_bootstrap_samples = 1000

# Create an array to store bootstrap sample means
bootstrap_sample_means = np.zeros(n_bootstrap_samples)

for i in range(n_bootstrap_samples):
    # Generate a bootstrap sample of paired differences
    bootstrap_sample = np.random.choice(test_predictions['model_difference'], len(test_predictions), replace=True)
    
    # Calculate the sample statistic (mean) for the bootstrap sample
    bootstrap_sample_means[i] = np.mean(bootstrap_sample)

# Calculate the confidence interval
confidence_interval = np.percentile(bootstrap_sample_means, [2.5, 97.5])

# Perform a t-test on the paired differences
ttest_statistic, ttest_pvalue = ttest_rel(test_predictions['y_pred_proba_xgboost'], test_predictions['y_pred_proba_ffnn'])
wilcoxon_pvalue = wilcoxon(test_predictions['y_pred_proba_xgboost'], test_predictions['y_pred_proba_ffnn'])
confidence_interval, ttest_pvalue, wilcoxon_pvalue

(array([0.05327034, 0.09534837]),
 6.0152950535030424e-12,
 WilcoxonResult(statistic=20177.0, pvalue=8.574266149430355e-11))

## extreme high evidence agiant null hypothesis ##

# q10: analyses for different model pairs on some different subsets of data #

In [ ]:
## fsi category

In [13]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel


country_indicators = pd.read_csv("country_indicators.csv")
test_predictions = pd.read_csv("test_predictions.csv")

# country == alert and country == stable
subset_alert = country_indicators[country_indicators['fsi_category'] == 'Alert']
subset_stable = country_indicators[country_indicators['fsi_category'] == 'Stable']

# data of country==alert and data of country== stable
data_subset_alert = test_predictions[test_predictions['iso3'].isin(subset_alert['iso3'])]
data_subset_stable = test_predictions[test_predictions['iso3'].isin(subset_stable['iso3'])]

# ttest
ttest_statistic, alert_ttest_pvalue = ttest_rel(data_subset_alert['y_pred_proba_transformer'], data_subset_alert['y_pred_proba_xgboost'])


ttest_statistic, stable_ttest_pvalue = ttest_rel(data_subset_stable['y_pred_proba_transformer'], data_subset_stable['y_pred_proba_xgboost'])
print(alert_ttest_pvalue, stable_ttest_pvalue)




6.26712025555292e-07 0.0024450270517777355
